In [ ]:
import xarray as xr
import cfgrib
from matplotlib.cm import get_cmap
import matplotlib.ticker as mticker
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import warnings
warnings.filterwarnings('ignore')
from wrf import smooth2d, getvar
# import os
# import h5py
from netCDF4 import Dataset
import matplotlib.transforms as mtransforms
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import griddata

def get_z(filepath, min_lat=-90, max_lat=90, min_lon=-180, max_lon=180):
    
    file = xr.open_dataset(filepath, engine='cfgrib',
                backend_kwargs={'filter_by_keys': {'typeOfLevel': 'isobaricInhPa','shortName': 'gh'}})
    var = file.get('gh')[10].to_dataframe()
    latitudes = var.index.get_level_values('latitude')
    longitudes = var.index.get_level_values('longitude')
    map_function = lambda lon: (lon - 360) if (lon > 180) else lon
    remapped_longitudes = longitudes.map(map_function)
    var['longitude'] = remapped_longitudes
    var['latitude'] = latitudes
    lat_filter = (var['latitude'] >= min_lat) & (var['latitude'] <= max_lat)
    lon_filter = (var['longitude'] >= min_lon) & (var['longitude'] <= max_lon)
    var = var.loc[lat_filter & lon_filter]
    var = var.set_index(['latitude', 'longitude']).to_xarray()
    
    return var

def get_t(filepath, min_lat=-90, max_lat=90, min_lon=-180, max_lon=180):
    
    file = xr.open_dataset(filepath, engine='cfgrib',
                backend_kwargs={'filter_by_keys': {'typeOfLevel': 'isobaricInhPa','shortName': 't'}})
    var = file.get('t')[10].to_dataframe()
    latitudes = var.index.get_level_values('latitude')
    longitudes = var.index.get_level_values('longitude')
    map_function = lambda lon: (lon - 360) if (lon > 180) else lon
    remapped_longitudes = longitudes.map(map_function)
    var['longitude'] = remapped_longitudes
    var['latitude'] = latitudes
    lat_filter = (var['latitude'] >= min_lat) & (var['latitude'] <= max_lat)
    lon_filter = (var['longitude'] >= min_lon) & (var['longitude'] <= max_lon)
    var = var.loc[lat_filter & lon_filter]
    var = var.set_index(['latitude', 'longitude']).to_xarray()
    
    return var

def create_lat_lon_grid(lat_range, lon_range, step=0.5):
    """
    Create 2D NumPy arrays of latitude and longitude points with given ranges.
    
    Args:
    - lat_range: Tuple of two floats representing the range of latitudes (min_lat, max_lat).
    - lon_range: Tuple of two floats representing the range of longitudes (min_lon, max_lon).
    - step: Float representing the step size in degrees for both latitude and longitude. Default is 0.25 degrees.
    
    Returns:
    - latitudes: 2D NumPy array of latitude points.
    - longitudes: 2D NumPy array of longitude points.
    """
    min_lat, max_lat = lat_range
    min_lon, max_lon = lon_range
    
    # Calculate the number of points in each dimension
    num_lats = int((max_lat - min_lat) / step) + 1
    num_lons = int((max_lon - min_lon) / step)
    
    # Generate arrays of latitude and longitude values
    latitudes = np.linspace(min_lat, max_lat, num_lats)
    longitudes = np.linspace(min_lon, max_lon, num_lons)
    
    # Create 2D grids of latitude and longitude points
    longitudes_grid, latitudes_grid = np.meshgrid(longitudes, latitudes)
    
    return latitudes_grid, longitudes_grid

# Example usage:
lat_range = (-90, 90)  # Latitude range from -90 to 90 degrees
lon_range = (-180, 180)  # Longitude range from -180 to 180 degrees
latsn, lonsn = create_lat_lon_grid(lat_range, lon_range)

In [ ]:
min_lat = -90
max_lat = 90
min_lon = -180
max_lon = 180

zg_full = get_z('/p/work1/lloveras/nov2018/gfs_files/analysis/gfs.0p25.2018111312.f000.grib2', min_lat, max_lat, min_lon, max_lon)
tg_full = get_t('/p/work1/lloveras/nov2018/gfs_files/analysis/gfs.0p25.2018111312.f000.grib2', min_lat, max_lat, min_lon, max_lon)

zg = np.asarray(zg_full['gh'])[::2,::2]
tg = np.asarray(tg_full['t'])[::2,::2]
latsg = np.asarray(tg_full['latitude'])[::2]
lonsg = np.asarray(tg_full['longitude'])[::2]

zn = np.asarray(np.reshape(np.fromfile('/p/work1/lloveras/nov2018/gfs_nav/N2018111312/geopht_pre_0600.0_0000.0_glob720x361_2018111312_00000000_fcstfld', dtype = '>f4',count=361*720), [361, 720]),dtype=np.float64)
tn = np.asarray(np.reshape(np.fromfile('/p/work1/lloveras/nov2018/gfs_nav/N2018111312/airtmp_pre_0600.0_0000.0_glob720x361_2018111312_00000000_fcstfld', dtype = '>f4',count=361*720), [361, 720]),dtype=np.float64)

zn = np.roll(zn,360,axis=1)
tn = np.roll(tn,360,axis=1) + 273.15

In [ ]:
### Contour interval
zlvls = np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])/5

### Plot subdomain
min_lat = 15
max_lat = 60
min_lon = -135
max_lon = -65

### Cartopy projection
proj = crs.LambertConformal(central_longitude=-100,standard_parallels=(20,70))

fig, axd = plt.subplot_mosaic([['left','cbar']],
                              constrained_layout=True, figsize=(7,4), dpi=200, 
                              gridspec_kw={'width_ratios':[1,0.05],'height_ratios':[1]},
                              per_subplot_kw={'left':{'projection':proj}})

trans = mtransforms.ScaledTranslation(1/30, -1/30, fig.dpi_scale_trans)

##############
### LEFT
#############

# Add the gridlines
gls = axd['left'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['left'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=True
gls.rotate_labels = False

cs1 = axd['left'].contour(lonsn, latsn, smooth2d(zn,2)/10.,
                  levels=np.arange(0,1500,15),
                  colors='k', transform=crs.PlateCarree())
axd['left'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,15),fontsize=6,colors='k')
cs2 = axd['left'].contour(lonsg, latsg, smooth2d(zg,2)/10.,
                  levels=np.arange(0,1500,15),
                  colors='magenta', transform=crs.PlateCarree())
axd['left'].clabel(cs2,fmt='%1.0f',inline=1,levels=np.arange(0,1500,15),fontsize=6,colors='magenta')
im1 = axd['left'].contourf(lonsn, latsn, tn - tg, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['left'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['left'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['left'].set_title('1200 UTC 13 Nov 2018 ($t$ = 0 h)')

### SET THE COLORBAR AND SHOW
cbar = fig.colorbar(im1, orientation='vertical', cax=axd['cbar'])
plt.savefig('/p/work1/lloveras/nov2018/figs_png/gfs_navgam_analysis.png',bbox_inches='tight')
plt.show()